In [1]:
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

import owlready2 as owl
from owlready2 import *
import types
owlready2.reasoning.JAVA_MEMORY = 200000

#import scipy
#from scipy.spatial import ConvexHull
#import cdd
#from cdd import RepType, Matrix, Polyhedron
#from fractions import Fraction

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from datetime import datetime
import random

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation




device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

In [2]:
# dir = '/Users/victorlacerda/Documents/VSCode/ELHFaithfulness/NormalizedOntologies/goslimyeast.xml.owl'
# dir = '/Users/victorlacerda/Documents/VSCode/ELHFaithfulness/NormalizedOntologies/galennorm.xml.owl'
dir = '/Users/victorlacerda/Documents/VSCode/ELHFaithfulness/NormalizedOntologies/gonorm.xml.owl'
# dir = '/Users/victorlacerda/Documents/VSCode/ELHFaithfulness/NormalizedOntologies/family_ontology.owl'

In [3]:
'''
Class for creating entities to
populate the creation of the
canonical models.

The .name attribute is used to
create a single representation
for concepts like A and B / 
B and A, as they are the same.
'''

class CanonicalModelElements:

    concept_names = {}
    concept_intersections = {}
    concept_restrictions = {}
    all_concepts = {}

    def __init__(self, concept):
        self.concept = concept
        self.name = self.get_name()
        self.get_element_dict()

    def get_name(self):

        # add \Top
        
        if type(self.concept) == ThingClass:
            return self.concept.name

        elif type(self.concept) == Restriction:
            return 'exists_' + self.concept.property.name + '.' + self.concept.value.name
        
        else:
            return 'And_' + ''.join(sorted(self.concept.Classes[0].name + self.concept.Classes[1].name)) # The name is sorted to avoid that (e.g) (A \and B) and (B \and A) are treated as different concepts
        
    def get_element_dict(self):

        if type(self.concept) == ThingClass:
            CanonicalModelElements.concept_names[self.name] = self
            CanonicalModelElements.all_concepts[self.name] = self

        elif type(self.concept) == Restriction:
            CanonicalModelElements.concept_restrictions[self.name] = self
            CanonicalModelElements.all_concepts[self.name] = self

        elif type(self.concept) == And:
            CanonicalModelElements.concept_intersections[self.name] = self
            CanonicalModelElements.all_concepts[self.name] = self

In [4]:
onto = get_ontology(dir)
onto = onto.load()

individuals = onto.individuals()
roles = onto.properties()
classes = onto.classes()
gcas = onto.general_class_axioms()

In [11]:
def get_canonical_model_elements(concept_names_iter, role_names_iter, ontology):
    
    onto = ontology

    for concept_name in concept_names_iter:
        CanonicalModelElements(concept_name)

    counter = 0

    for _, value in CanonicalModelElements.concept_names.items():
        for _, concept2 in CanonicalModelElements.concept_names.items():
        
            with onto:
                gca = GeneralClassAxiom(value.concept & concept2.concept)
                gca.is_a.append(value.concept & concept2.concept)
            
            CanonicalModelElements(gca.left_side)

        counter += 1
        if counter%1 == 0:
            print(f'{counter}')

    print('')
    print('===========================================================================================================')
    print('All Concept Names and Concept Intersections have been preprocessed for the creation of the canonical model.')
    print('===========================================================================================================')

    #concept_names_iter.append(top)
    #concept_names_iter.append(bottom)

    for role_name in list(role_names_iter):
        for _, value in CanonicalModelElements.concept_names.items():

            with onto:
                gca = GeneralClassAxiom(role_name.some(value.concept))
                gca.is_a.append(role_name.some(value.concept))

            CanonicalModelElements(gca.left_side)

    print('')
    print('')
    print('All restrictions have been preprocessed for the creation of the canonical model.')

In [12]:
onto = get_ontology(dir)
onto = onto.load()

In [13]:
get_canonical_model_elements(classes, roles, onto)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53


KeyboardInterrupt: 

In [ ]:
len(CanonicalModelElements.concept_names.keys()), len(CanonicalModelElements.concept_intersections.keys())

In [ ]:
len(CanonicalModelElements.concept_names.keys()), len(CanonicalModelElements.concept_intersections.keys()), len(CanonicalModelElements.concept_restrictions.keys())

In [ ]:
'''
Class for creating entities to
populate the creation of the
canonical models.

The .name attribute is used to
create a single representation
for concepts like A and B / 
B and A, as they are the same.
'''

class CanonicalModelElements:

    concept_names = {}
    concept_intersections = {}
    concept_restrictions = {}
    all_concepts = {}

    def __init__(self, concept):
        self.concept = concept
        self.name = self.get_name()
        self.get_element_dict()

    def get_name(self):

        # add \Top
        
        if type(self.concept) == ThingClass:
            return self.concept.name

        elif type(self.concept) == Restriction:
            return 'exists_' + self.concept.property.name + '.' + self.concept.value.name
        
        else:
            return 'And_' + ''.join(sorted(self.concept.Classes[0].name + self.concept.Classes[1].name)) # The name is sorted to avoid that (e.g) (A \and B) and (B \and A) are treated as different concepts
        
    def get_element_dict(self):

        if type(self.concept) == ThingClass:
            CanonicalModelElements.concept_names[self.name] = self
            CanonicalModelElements.all_concepts[self.name] = self

        elif type(self.concept) == Restriction:
            CanonicalModelElements.concept_restrictions[self.name] = self
            CanonicalModelElements.all_concepts[self.name] = self

        elif type(self.concept) == And:
            CanonicalModelElements.concept_intersections[self.name] = self
            CanonicalModelElements.all_concepts[self.name] = self

In [ ]:
def get_canonical_model_elements(concept_names_iter, role_names_iter, ontology):
    
    onto = ontology
    top = owl.Thing
    bottom = owl.Nothing

    CanonicalModelElements(top)
    CanonicalModelElements(bottom)

    for concept_name in concept_names_iter:
        
        CanonicalModelElements(concept_name)
        for concept_name2 in concept_names_iter:
        
            with onto:
                gca = GeneralClassAxiom(concept_name & concept_name2)
                gca.is_a.append(concept_name & concept_name2)
            
            CanonicalModelElements(gca.left_side)

    print('')
    print('===========================================================================================================')
    print('All Concept Names and Concept Intersections have been preprocessed for the creation of the canonical model.')
    print('===========================================================================================================')

    concept_names_iter.append(top)
    #concept_names_iter.append(bottom)

    for role_name in role_names_iter:
        for concept_name in concept_names_iter:
            with onto:
                gca = GeneralClassAxiom(role_name.some(concept_name))
                gca.is_a.append(role_name.some(concept_name))

            CanonicalModelElements(gca.left_side)

    print('')
    print('')
    print('All restrictions have been preprocessed for the creation of the canonical model.')

In [ ]:
'''
The main class for creating the canonical model for the ontology.

The canonical model is stored in dictionaries available as class variables 'concept_canonical_interpretation'
and 'role_canonical_interpretation'. 

Args:
    concept_names_dict: a dictionary stored in the CanonicalModelElement class.
    concept_intersection_dict: a dictionary stored in the CanonicalModelElement class.
    concept_restrictions_dict: a dictionary stored in the CanonicalModelElement class.
    all_concepts_dict: a dictionary stored in the CanonicalModelElement class.
    role_names_iter (list): a list containing all role names in the loaded ontology.
'''

class CanonicalModel:

    concept_canonical_interpretation = {}
    role_canonical_interpretation = {}

    def __init__(self, concept_names_dict, concept_intersections_dict, concept_restrictions_dict, all_concepts_dict, role_names_iter):
        
        self.domain = all_concepts_dict
        self.concept_names_dict = concept_names_dict
        self.concept_restrictions_dict = concept_restrictions_dict
        self.concept_intersections_dict = concept_intersections_dict

        self.role_names_iter = role_names_iter

        self.concept_canonical_interp = self.get_concept_name_caninterp() # These are only used to build the concept_canonical_interpretation and role_canonical_interpretation class attributes
        self.role_canonical_interp = self.get_role_name_caninterp()       # The functions do not return anything, they just update their corresponding class variables 

    def get_concept_name_caninterp(self):

        # The variable concept is a string containing the name of an element of the domain of the canonical model
        # The key to the concept_names_dict variable corresponds to concept.name
        # This name can be used to access the concept in owlready2's format

        for concept in self.concept_names_dict.keys():

            CanonicalModel.concept_canonical_interpretation[concept] = []
            superclasses = self.domain[concept].concept.ancestors(include_self=True, include_constructs=True) # The self.domain[concept] is used to access the CanonicalModelElements type of object,
                                                                                                               # and the attribute .concept is used to access the concept in owlready2 format
                                                                                                              
            for superclass in superclasses:

                if type(superclass) == ThingClass:
                    CanonicalModel.concept_canonical_interpretation[concept].append(superclass.name)

                elif type(superclass) == Restriction:
                    CanonicalModel.concept_canonical_interpretation[concept].append('exists_' + superclass.property.name + '.' + superclass.value.name)

                elif type(superclass) == And:
                    if 'And_' + ''.join(sorted(superclass.Classes[0].name + superclass.Classes[1].name)) in CanonicalModel.concept_canonical_interpretation[concept]:
                        pass
                    else:
                        CanonicalModel.concept_canonical_interpretation[concept].append('And_' + ''.join(sorted(superclass.Classes[0].name + superclass.Classes[1].name)))

            
    def get_role_name_caninterp(self):

        # Second case from Definition 10

        for role_name in self.role_names_iter:

            role_name_str = role_name.name # Accesses the property type object's name as a string
            CanonicalModel.role_canonical_interpretation[role_name_str] = []

            for restriction_name in self.concept_restrictions_dict.keys(): # Where restriction_name denotes a \exists r.B type of concept 'exists_' + self.concept.property.name + '.' + self.concept.value.name
                c_B = self.concept_restrictions_dict[restriction_name].concept.value.name

                if role_name_str in restriction_name:
                    
                    superclasses = self.domain[restriction_name].concept.ancestors(include_self=True, include_constructs=False) # Include_constructs is turned to false due to the definition of canonical model

                    for superclass in superclasses:
                        super_superclasses = superclass.ancestors(include_self=True, include_constructs=True)

                        for super_superclass in super_superclasses:

                            if type(super_superclass) == ThingClass:
                                c_D = super_superclass.name
                                CanonicalModel.role_canonical_interpretation[role_name_str].append((c_D, c_B))

                            elif type(super_superclass) == Restriction:
                                c_D = 'exists_' + super_superclass.property.name + '.' + super_superclass.value.name
                                CanonicalModel.role_canonical_interpretation[role_name_str].append((c_D, c_B))

                            elif type(super_superclass) == And:
                                c_D = 'And_' + ''.join(sorted(super_superclass.Classes[0].name + super_superclass.Classes[1].name))
                                CanonicalModel.role_canonical_interpretation[role_name_str].append((c_D, c_B))

        # First case from Definition 10

        for restriction_name in self.concept_restrictions_dict.keys():

            concept_name_str = self.concept_restrictions_dict[restriction_name].concept.value.name
            role_name_str = self.concept_restrictions_dict[restriction_name].concept.property.name

            role_name = self.concept_restrictions_dict[restriction_name].concept.property

            superroles = list(role_name.ancestors(include_self=True))
            
            pair = (restriction_name, concept_name_str)

            for superrole in superroles:
                if superrole.name in CanonicalModel.role_canonical_interpretation.keys():
                    CanonicalModel.role_canonical_interpretation[superrole.name].append(pair)
                else:
                    pass

In [ ]:
'''
Main function for creating the canonical model.

    Args:
        onto_dir (str): a string pointing to the directory where the ontology is stored.

    Returns:
        canmodel (CanonicalModel): returns a variable containing the canonical model. 
        
        Attention: the interpretations of concept names and role names can also be accessed via class variables
        from the CanonicalModel class.
'''

def create_canonical_model(onto_dir):

    onto = get_ontology(onto_dir)
    onto = onto.load()

    individuals_iter = list(onto.individuals())

    gcas_iter = list(onto.general_class_axioms()) # Attention: this will not work unless the generator is converted into a list
    concept_names_iter = list(onto.classes())
    role_names_iter = list(onto.properties())
    individuals_iter = list(onto.individuals())

    # preprocess_namespaces(onto, individuals_iter, role_names_iter, concept_names_iter)


    get_canonical_model_elements(concept_names_iter, role_names_iter, onto)

    print('============================================================================')
    print('Starting to reason.\n')

    with onto:
        sync_reasoner()
        
    #onto.save("inferences_goslimyeast.owl")

    gcas_iter = list(onto.general_class_axioms()) # Attention: this will not work unless the generator is converted into a list
    concept_names_iter = list(onto.classes())
    role_names_iter = list(onto.properties())
    individuals_iter = list(onto.individuals())

    print('')
    print('============================================================================')
    print('Done reasoning. Creating the canonical model.')
    canmodel = CanonicalModel(CanonicalModelElements.concept_names, CanonicalModelElements.concept_intersections, CanonicalModelElements.concept_restrictions, CanonicalModelElements.all_concepts, role_names_iter)
    print('============================================================================\n')
    print('Concluded creating canonical model.')

    return canmodel

In [ ]:
# Instantiates the canonical model

canmodel = create_canonical_model(dir)